In [2]:
import sys
sys.path.append('/content/drive/Othercomputers/내 iMac/Meta_Learning/코스웍/2022 고급기계학습주제 (김광인)/과제/CW1')

In [3]:
from utils.utils import *
from algorithms.random_forest import RandomForest

In [4]:
import numpy as np
from sklearn.model_selection import ParameterGrid
from sklearn.metrics import accuracy_score

In [5]:
# seed fixation
np.random.seed(1)

In [6]:
# path for colab only
import os
colab = '/content/drive/Othercomputers/내 iMac/Meta_Learning/코스웍/2022 고급기계학습주제 (김광인)/과제/CW1/data'
PATH1 = os.path.join(colab, 'cifar-10-batches-py/data_batch_1')
PATH2 = os.path.join(colab, 'cifar-100-python/train')

# Load data
cifar10 = unpickle(PATH1)
cifar100 = unpickle(PATH2)

# Generate dataset
cifar10 = dataset_generator(cifar10, 3000, cifar_type='cifar10')
cifar100 = dataset_generator(cifar100, 3000, cifar_type='cifar100')

# test path
PATH1_t = os.path.join(colab, 'cifar-10-batches-py/data_batch_2')
PATH2_t = os.path.join(colab, 'cifar-100-python/test')

# Load test data
test_cifar10 = unpickle(PATH1_t)
test_cifar100 = unpickle(PATH2_t)

# Generate test dataset
test_cifar10 = dataset_generator(test_cifar10, 300, cifar_type='cifar10')
test_cifar100 = dataset_generator(test_cifar100, 300, cifar_type='cifar100')

# Normalise
cifar10['data'] = cifar10['data']/255
cifar100['data'] = cifar100['data']/255

test_cifar10['data'] = test_cifar10['data']/255
test_cifar100['data'] = test_cifar100['data']/255

In [7]:
X10 = cifar10['data']
y10 = np.array(cifar10['labels'])
X100 = cifar100['data']
y100 = np.array(cifar100['labels'])

test_X10 = test_cifar10['data']
test_y10 = np.array(test_cifar10['labels'])
test_X100 = test_cifar100['data']
test_y100 = np.array(test_cifar100['labels'])

## Hyperparameter Search: Train/Test Set

### cifar10
According to DecisionTree analysis,<br>
__cifar10 best model__: {'max_depth': 100, 'max_features': 55, 'min_sample_split': 100}, accuracy: __0.28__<br>
__cifar100 best model__: {'max_depth': 100, 'max_features': 200, 'min_sample_split': 30} accuracy: __0.063__

In [8]:
param_grid = {
    'max_depth': [100],
    'min_sample_split': [100],
    'max_features': [55],
    'n_trees': [3, 5, 10]
}

grid = list(ParameterGrid(param_grid))

In [9]:
grid

[{'max_depth': 100, 'max_features': 55, 'min_sample_split': 100, 'n_trees': 3},
 {'max_depth': 100, 'max_features': 55, 'min_sample_split': 100, 'n_trees': 5},
 {'max_depth': 100,
  'max_features': 55,
  'min_sample_split': 100,
  'n_trees': 10}]

In [ ]:
import time

train_time = []
inf_time = []
acc = []

for i, args in enumerate(grid):
    print(args)
    clf = RandomForest(**args)
    
    # check time
    start = time.time()
    
    print(f'{i}th classifier training...', end='')
    clf.fit(X10, y10)
    
    end1 = time.time()
    train_time.append(end1-start)
    print( 'time: ', round(end1-start,1), end='')

    y_pred = clf.predict(test_X10)
    ac = accuracy_score(test_y10, y_pred)
    print(' acc: ', ac)

    end2 = time.time()
    inf_time.append(end2-end1)
    acc.append(ac)

# Rounding
tt = [round(t,1) for t in train_time]
it = [round(t,4) for t in inf_time]
ac = [round(a,3) for a in acc]

trial = {'train_time': tt, 'inference_time': it, 'accuracy': ac}

import json

with open("RF_cifar10.json", "w") as json_file:
    json.dump(trial, json_file)

print()
for args, a in zip(grid, ac):
    print(args, a)

{'max_depth': 100, 'max_features': 55, 'min_sample_split': 100, 'n_trees': 3}
0th classifier training...